In [ ]:
# Import required libraries
import mysql.connector
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import time

# Load environment variables
load_dotenv()

# Retrieve MySQL credentials securely
mysql_host = os.getenv("MYSQL_HOST", "127.0.0.1")
mysql_port = int(os.getenv("MYSQL_PORT", 3306))
mysql_user = os.getenv("MYSQL_USER")
mysql_password = os.getenv("MYSQL_PASSWORD")
mysql_database = os.getenv("MYSQL_DATABASE")
mysql_use_pure = os.getenv("MYSQL_USE_PURE", "True").lower() == "true"

# Connect to MySQL
try:
    conn = mysql.connector.connect(
        host=mysql_host,
        port=mysql_port,
        user=mysql_user,
        password=mysql_password,
        database=mysql_database,
        use_pure=mysql_use_pure
    )
    cursor = conn.cursor()
    print("Connected Successfully to MySQL")
except mysql.connector.Error as err:
    print(f"Connection failed: {err}")
    exit()  # Stop script if connection fails

# Fetch all marketable item IDs from XIVAPI
def fetch_all_items():
    xivapi_url = "https://xivapi.com/item"
    all_items = []
    page = 1

    print("Starting to fetch marketable items from XIVAPI...")

    while True:
        response = requests.get(xivapi_url, params={"limit": 1000, "page": page})
        print(f"Fetching XIVAPI Page {page} - Status Code: {response.status_code}")

        if response.status_code != 200:
            print(f"Error: XIVAPI Request Failed with Status Code {response.status_code}")
            return []

        data = response.json()

        if "Results" not in data:
            print("Warning: 'Results' key is missing from API response. Stopping...")
            break  # Stop if no results are returned

        for item in data["Results"]:
            item_id = item["ID"]
            item_name = item.get("Name", "Unknown Item")

            # Fetch full item details
            item_details_url = f"https://xivapi.com/item/{item_id}"
            item_response = requests.get(item_details_url)

            # Fix: Check if the response is empty or failed
            if item_response.status_code != 200 or not item_response.text.strip():
                print(f"Warning: Empty response for item {item_id}, skipping...")
                continue  # Skip this item

            item_response = item_response.json()  # Now it's safe to parse JSON

            # Extract category
            item_category = "Unknown Category"
            if "ItemSearchCategory" in item_response and item_response["ItemSearchCategory"]:
                item_category = item_response["ItemSearchCategory"].get("Name", "Unknown Category")

            # Extract crafting class (check both `ClassJobRepair` and `ClassJobCategory`)
            crafting_class = "None"
            if "ClassJobRepair" in item_response and item_response["ClassJobRepair"]:
                crafting_class = item_response["ClassJobRepair"].get("Name", "None")
            elif "ClassJobCategory" in item_response and item_response["ClassJobCategory"]:
                crafting_class = item_response["ClassJobCategory"].get("Name", "None")

            all_items.append((item_id, item_name, item_category, crafting_class))

        # Show progress update with total items fetched so far
        total_items_fetched = len(all_items)
        estimated_total_pages = data["Pagination"].get("PageTotal", "Unknown")

        print(f"Page {page}: Retrieved {len(data['Results'])} items (Total items so far: {total_items_fetched})")
        print(f"Estimated total pages: {estimated_total_pages}")

        # Check if there's another page of results
        if not data["Pagination"]["PageNext"]:
            break

        page += 1
        time.sleep(0.5)  # Avoid hitting API rate limits

    print(f"Finished fetching items! Total items retrieved: {len(all_items)}")
    return all_items

# Insert item data into MySQL with real item names
def insert_items(cursor, conn, all_items):
    print(f"Inserting {len(all_items)} items into MySQL...")

    for item_id, item_name, item_category, crafting_class in all_items:
        cursor.execute(
            """
            INSERT INTO items (item_id, item_name, item_category, crafting_class)
            VALUES (%s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                item_name = VALUES(item_name),
                item_category = VALUES(item_category),
                crafting_class = VALUES(crafting_class)
            """,
            (item_id, item_name, item_category, crafting_class)
        )

    conn.commit()
    print("All items inserted into MySQL!")

# Fetch all items from XIVAPI
all_items = fetch_all_items()

# Insert items into MySQL
insert_items(cursor, conn, all_items)

# Close MySQL Connection
cursor.close()
conn.close()
print("MySQL connection closed.")
